In [30]:
import numpy as np
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, GRU, Dropout, Dense


def predict_next_day_values(data_path, get_value: str):
    # Load data from JSON file
    with open(data_path) as f:
        data = json.load(f)

    # Extract features and target
    X = np.column_stack((data["o"], data["h"], data["l"], data["c"], data["v"]))
    y = np.array(data[get_value])  # Close price

    # Normalize data
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

    # Reshape data for LSTM input (samples, timesteps, features)
    X_scaled = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_scaled, test_size=0.3, random_state=42
    )

    # Build the model
    model = Sequential()
    model.add(
        Bidirectional(
            GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1))
        )
    )
    model.add(Dropout(0.2))
    model.add(Bidirectional(GRU(units=50)))
    model.add(Dropout(0.2))
    model.add(Dense(32, kernel_initializer="uniform", activation="relu"))
    model.add(Dense(1, kernel_initializer="uniform", activation="linear"))
    model.compile(optimizer="adam", loss="mean_squared_error")

    # Train the model
    model.fit(X_train, y_train, epochs=3, batch_size=32, verbose=0)

    # Predict tomorrow's closing price
    last_data_point = X_scaled[-1].reshape(1, X_scaled.shape[1], 1)
    predicted_price_scaled = model.predict(last_data_point)
    predicted_price = scaler_y.inverse_transform(predicted_price_scaled)

    return predicted_price


# Example usage:
predicted_price = predict_next_day_values("data.json", "c")
print("Predicted closing price for tomorrow:", predicted_price)

Epoch 1/10
126/126 [==============================] - 10s 35ms/step - loss: 0.0251
Epoch 2/10
126/126 [==============================] - 3s 23ms/step - loss: 3.3088e-04
Epoch 3/10
126/126 [==============================] - 3s 25ms/step - loss: 1.7352e-04
Epoch 4/10
126/126 [==============================] - 3s 24ms/step - loss: 1.1083e-04
Epoch 5/10
126/126 [==============================] - 3s 23ms/step - loss: 8.3246e-05
Epoch 6/10
126/126 [==============================] - 4s 28ms/step - loss: 6.4428e-05
Epoch 7/10
126/126 [==============================] - 4s 30ms/step - loss: 4.9193e-05
Epoch 8/10
126/126 [==============================] - 3s 24ms/step - loss: 3.9890e-05
Epoch 9/10
126/126 [==============================] - 3s 25ms/step - loss: 3.7736e-05
Epoch 10/10
1/1 [==============================] - 1s 1s/step
Predicted closing price for tomorrow: [[407021.03]]


In [20]:
# Example usage:
predictor = StockPredictor("data.json")

In [ ]:
X_train, X_test, y_train, y_test = predictor.preprocess_data("c")
predictor.build_model()
predictor.train_model(X_train, y_train, epochs=100)
predicted_price = predictor.predict_next_day_value(
    X_test[-1].reshape(1, X_test.shape[1], 1)
)
print("Predicted closing price for tomorrow:", predicted_price)

In [12]:
# Save model
predictor.save_model("stock_model.h5")

/Users/khanhle/miniforge3/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
# Load model
predictor.load_model("stock_model.h5")

In [15]:
predicted_price_loaded = predictor.predict_next_day_value(
    X_test[-1].reshape(1, X_test.shape[1], 1)
)

1/1 [==============================] - 0s 74ms/step


In [16]:
print("Predicted closing price for tomorrow (loaded model):", predicted_price_loaded)

Predicted closing price for tomorrow (loaded model): [[928.90393]]
